# Winston Lutz from Varian DICOM images

Here is a notebook demonstrating pulling images out of DICOM files, finding both the field position and the BB position within it.


## Disclaimer

All pixel size extraction code given here has not been appropriately validated.
I do not have access to a Varian Linac. Validation of this example from a Varian user would be appreciated. Please raise an issue via https://github.com/pymedphys/pymedphys/issues/new to either confirm that you have been able to validate this or provide a counter example to where there are issues here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import IPython.display

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
import pymedphys.wlutz

## Initialising field penumbra and ball bearing diameter

The `pymedphys` code aims to match opposing penumbras of the field. To do this it uses a penumbra width parameter. A value of 2 (mm) is usually sufficient here. The `bb_diameter` parameter is used to define the diameter of the ball bearing used within the image.

In [ ]:
penumbra = 2
bb_diameter = 8

## Download some example files

For our purpose here, let's download some files. They will download into a `.pymedphys` directory within your home drive.

In [ ]:
dicom_wlutz_paths = pymedphys.zip_data_paths('denis_wlutz_images.zip')
dicom_wlutz_paths

In [ ]:
dicom_files = [pydicom.read_file(str(path), force=True) for path in dicom_wlutz_paths]

## Defining a grid and field size extraction function

So as to appropriately scale the image the pixel scaling is extracted from the DICOM header. This is the part of this example that has not undergone appropriate validation. The field size is also extracted from the DICOM header here.

In [ ]:
def get_parameters_from_dicom(dicom_header):
    sad = float(dicom_header.RadiationMachineSAD)
    panel_adjustment = -float(dicom_header.XRayImageReceptorTranslation[2])
    panel_ssd = panel_adjustment + sad

    guess_at_pixel_spacing_at_iso = np.array(dicom_header.ImagePlanePixelSpacing).astype(float) / panel_ssd * sad
    dx, dy = guess_at_pixel_spacing_at_iso
    
    half_range_x = dicom_header.Columns * dy / 2
    half_range_y = dicom_header.Rows * dx / 2
    
    x = np.linspace(-half_range_x, half_range_x, dicom_header.Columns)
    y = np.linspace(-half_range_y, half_range_y, dicom_header.Rows)
    
    jaw_pos = {
        coll.RTBeamLimitingDeviceType: np.array(coll.LeafJawPositions).astype(float)
        for coll in dicom_header.ExposureSequence[0].BeamLimitingDeviceSequence
    }

    field_size_x = np.diff(jaw_pos['ASYMX'])[0]
    field_size_y = np.diff(jaw_pos['ASYMY'])[0]

    edge_lengths = [field_size_x, field_size_y]
    
    return x, y, edge_lengths


get_parameters_from_dicom(dicom_files[0])

## Extracting Gantry and Collimator angles

So as to report the gantry and collimator positions for each image they are extracted.

In [ ]:
gantries = np.array([
    np.round(dcm.GantryAngle, 2) for dcm in dicom_files
])

gantries

In [ ]:
colls = np.array([
    np.round(dcm.BeamLimitingDeviceAngle, 2) for dcm in dicom_files
])

colls

## Running the analysis

Here each image is analysed and reported. By seeing the overlay of the field and bb on the image, as well as seeing the profiles flipped about the reported respective centres it is possible to visually validate that the software has appropriately (or not) found the correct locations in each image.

In [ ]:
def display_markdown(string):
    IPython.display.display(IPython.display.Markdown(string))

In [ ]:
for dcm, coll, gantry in zip(dicom_files, colls, gantries):
    img = dcm.pixel_array[::-1, :] / 2 ** 16 
    x, y, edge_lengths = get_parameters_from_dicom(dcm)
    
    display_markdown(f'## Gantry {coll} | Collimator {gantry}')
    
    bb_centre, field_centre, field_rotation = pymedphys.wlutz.find_field_and_bb(
        x,
        y,
        img,
        edge_lengths,
        bb_diameter,
        penumbra=2,
        fixed_rotation=coll,
        pylinac_tol=0.2
    )   

    pymedphys.wlutz.reporting( 
        x,
        y,
        img,
        bb_centre,
        field_centre,
        field_rotation,
        bb_diameter,
        edge_lengths,
        penumbra,
    )
    
    deviation = np.round(np.array(field_centre) - np.array(bb_centre), 2)
    
    display_markdown(
        f'PyMedPhys field centre - BB centre (mm):\n\n```python\n[x, y] = [{deviation[0]}, {deviation[1]}]\n```'
    )
    plt.show()